In [1]:
# Mount google drive
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Import libraries and ignore warnings
import warnings
warnings.filterwarnings('ignore')
from tensorflow import set_random_seed
import pandas as pd
import numpy as np
np.random.seed(2)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Plot architecture
import keras
import pydot
from keras.utils.vis_utils import model_to_dot

from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

import os
from PIL import Image

Using TensorFlow backend.


In [0]:
# Read the annotations file that contains the label and the image file name
labels = pd.read_csv('gdrive/My Drive/Colab Notebooks/Computer Vision/SMILE_Dataset/annotations.csv', header=None, names=['fname','label'])

# Shuffle data
labels = labels.sample(frac=1).reset_index()

# Use a list comprehension to loop over image file names and import one by one and store pixel values
x = np.array([image.img_to_array(image.load_img('gdrive/My Drive/Colab Notebooks/Computer Vision/SMILE_Dataset/all/'+fname[:-3]+'jpg', target_size=(96, 96))) for fname in labels['fname']])

# Because the names are strings, the neural network only takes in numerical formats so we will one-hot encode the label
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['label'])
y = integer_encoded

In [0]:
def akshay():
    
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(96, 96, 3)))
    model.add(layers.Dense(80, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

In [0]:
def last_time():
    """
    """
    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.1))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

In [0]:
def sota_1():
    """
    Reaches about 96%. This is the best architecture so far.
    """
    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  #optimizer=optimizers.RMSprop(lr=1e-4), 91.93%
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

In [92]:
# All classification reports will be added here. When we are done we can average the f1 scores
reports = []
np.random.seed(2) # numpy seed generator
set_random_seed(2) # Tensorflow seed generator
# Apply stratified K-fold ith 10 splits. Stratified means the same distribution of classes than the whole dataset
# In this case, 50-50
kf = StratifiedKFold(n_splits=10)


# Just for printing purposes
id = 1

for train_index, test_index in kf.split(x,y):
    np.random.seed(2) # numpy seed generator
    set_random_seed(2) # Tensorflow seed generator

    print('Kfold iteration {}/10'.format(id))
    print('Total images: {} ---- Train images: {} ---- Test images: {}'.format(len(x),len(train_index),len(test_index)))
    id += 1 

    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

    

    # We identified some optimal affine transformations
    datagen1 = ImageDataGenerator(rescale=1./255,
                                 rotation_range=20, # randomly rotate images in the range (degrees, 0 to 180)
                                 width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
                                 height_shift_range=0.1, 
                                 shear_range=0.1,
                                 zoom_range=0.2)

    datagen1.fit(X_train)

    datagen2 = ImageDataGenerator(rescale=1./255,
                                 rotation_range=40, # randomly rotate images in the range (degrees, 0 to 180)
                                 width_shift_range=0.2, # randomly shift images horizontally (fraction of total width)
                                 height_shift_range=0.2, 
                                 shear_range=0.3,
                                 zoom_range=0.4)

    datagen2.fit(X_train)

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

    # Ensemble Learning
    ensemble_predictions = {}
    for i in range(2):
        print("training model:",i)
        model = last_time()
        history = model.fit_generator(datagen1.flow(X_train, y_train, batch_size = 16), epochs = 30, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 16,
                              callbacks=[learning_rate_reduction])
        
        y_pred_test = model.predict(X_test)
        y_pred_test = y_pred_test.astype(int) # Convert float to int
        y_pred_test = y_pred_test.reshape(y_pred_test.shape[0]) # Reshape

        ensemble_predictions['last_time_'+str(i)] = y_pred_test

    for i in range(2):
        print("training model:",i)
        model = akshay()
        history = model.fit_generator(datagen1.flow(X_train, y_train, batch_size = 16), epochs = 20, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 16,
                              callbacks=[learning_rate_reduction])
        
        y_pred_test = model.predict(X_test)
        y_pred_test = y_pred_test.astype(int) # Convert float to int
        y_pred_test = y_pred_test.reshape(y_pred_test.shape[0]) # Reshape

        ensemble_predictions['akshay_'+str(i)] = y_pred_test

    for i in range(3):
        print("training model:",i)
        model = sota_1()
        history = model.fit_generator(datagen1.flow(X_train, y_train, batch_size = 16), epochs = 20, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 16,
                              callbacks=[learning_rate_reduction])
        
        y_pred_test = model.predict(X_test)
        y_pred_test = y_pred_test.astype(int) # Convert float to int
        y_pred_test = y_pred_test.reshape(y_pred_test.shape[0]) # Reshape

        ensemble_predictions['sota_1_'+str(i)] = y_pred_test

    ensemble_df = pd.DataFrame(ensemble_predictions)

    temp_test = ensemble_df.mode(axis=1)[0]

    #print(classification_report(y_test, y_pred))
    reports.append(classification_report(y_test, temp_test,output_dict=True))

    

Kfold iteration 1/10
Total images: 400 ---- Train images: 360 ---- Test images: 40
training model 1
training model: 0
Epoch 1/30
23/22 [==============================] - 9s 403ms/step - loss: 0.6972 - acc: 0.5221 - val_loss: 1.6480 - val_acc: 0.5000
Epoch 2/30
23/22 [==============================] - 1s 39ms/step - loss: 0.6489 - acc: 0.5952 - val_loss: 4.3842 - val_acc: 0.7250
Epoch 3/30
23/22 [==============================] - 1s 56ms/step - loss: 0.5213 - acc: 0.7583 - val_loss: 2.7903 - val_acc: 0.8250
Epoch 4/30
23/22 [==============================] - 1s 54ms/step - loss: 0.3255 - acc: 0.8586 - val_loss: 1.5942 - val_acc: 0.9000
Epoch 5/30
23/22 [==============================] - 1s 56ms/step - loss: 0.3149 - acc: 0.8696 - val_loss: 2.3914 - val_acc: 0.8500
Epoch 6/30
23/22 [==============================] - 1s 55ms/step - loss: 0.2831 - acc: 0.8996 - val_loss: 1.1669 - val_acc: 0.9250
Epoch 7/30
23/22 [==============================] - 1s 57ms/step - loss: 0.2362 - acc: 0.9048 -

In [93]:
# We loop over all reports (1 per fold) and then compute the average of all weighted f1 scores
final_f1_score = np.mean([rep['weighted avg']['f1-score'] for rep in reports])

print('Final F1-Score is: {}%'.format(np.round(final_f1_score*100,2)))

Final F1-Score is: 97.25%
